In [9]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from psql_helpers import read_table_async

In [11]:
from psql_models import *

In [12]:
from psql_access import get_meeting_token
from datetime import datetime
from indexing.redis_keys import RedisKeys
from indexing.worker import IndexingWorker
from redis import Redis


In [13]:
from sqlalchemy import update
from psql_helpers import get_session

async def reset_content_indexing():
    async with get_session() as session:
        await session.execute(
            update(Content).values(is_indexed=False)
        )
        await session.commit()

# Execute
await reset_content_indexing()

In [14]:
df = await read_table_async(Content)

In [15]:
df.sort_values('timestamp', ascending=False)

,id,type,text,timestamp,parent_id,is_indexed,last_update
17,60a35cd1-f7cc-4068-a454-ca94ea2d2154,meeting,None,2025-01-27 17:36:50,None,False,2025-01-27 17:52:33
16,a2f51995-da55-47b3-801a-fb4f60552c46,meeting,None,2025-01-27 17:33:25,None,False,2025-01-27 17:52:29
4510,e413587c-2146-438b-a3a6-50cf2455bada,meeting,None,2025-01-27 17:17:52,None,False,2025-01-27 17:22:08
4513,1b35a663-436e-4de7-b81a-941ab0875a1a,meeting,None,2025-01-27 17:03:40,None,False,2025-01-27 17:52:33
4507,5aef356f-2027-4970-8932-abf1780159b3,meeting,None,2025-01-27 17:02:59,None,False,2025-01-27 17:11:35
...,...,...,...,...,...,...,...
4502,5179ee46-39df-4a3c-a34e-edff67fa5ef1,title,16:30,NaT,ca517d70-323b-4f01-b41b-cc1c9fdfc911,False,NaT
4503,01f56681-c718-4e0a-899a-03b8f673f760,title,17:02,NaT,5aef356f-2027-4970-8932-abf1780159b3,False,NaT
4505,77f768ba-8374-4977-8d3c-2b164df65852,title,17:01,NaT,3bf4c25e-be49-4e24-8288-12ef3c8c9218,False,NaT
4506,39d1f969-1ab5-445c-89e9-cf2370eec350,title,17:03,NaT,1b35a663-436e-4de7-b81a-941ab0875a1a,False,NaT


In [16]:
df[df['is_indexed']]

,id,type,text,timestamp,parent_id,is_indexed,last_update


In [8]:
df = await read_table_async(UserContent)

In [9]:
df

,id,content_id,user_id,access_level,created_at,created_by,is_owner
0,1,dfa86d3e-4946-414d-bd66-bd0ea472dc5c,52a33c88-4972-45ac-ac31-000bdd0799fb,owner,2024-06-05 12:56:23.602416+00:00,52a33c88-4972-45ac-ac31-000bdd0799fb,True
1,2,f858cb3e-ac65-467c-8f5f-d0704abff5d3,d5fe11cf-b87e-4bf8-9800-d8931098bfb2,owner,2024-06-18 11:00:08.984681+00:00,d5fe11cf-b87e-4bf8-9800-d8931098bfb2,True
2,3,f858cb3e-ac65-467c-8f5f-d0704abff5d3,8b767b85-44fb-4c20-97df-7aa172242273,owner,2024-06-18 11:00:08.984681+00:00,8b767b85-44fb-4c20-97df-7aa172242273,True
3,4,921b32ba-b5d1-4770-bc90-2a92de118a2d,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-06-13 18:44:16.168998+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
4,5,f57553c1-1b73-47e6-97e9-ae3d5295d6d4,d5fe11cf-b87e-4bf8-9800-d8931098bfb2,owner,2024-06-10 15:09:50.271597+00:00,d5fe11cf-b87e-4bf8-9800-d8931098bfb2,True
...,...,...,...,...,...,...,...
4777,4778,7e332121-e842-4b09-b2a9-b97013de88e5,12262e9c-a9e8-49b3-b0ee-547240fac7a9,owner,2025-01-20 13:04:44+00:00,12262e9c-a9e8-49b3-b0ee-547240fac7a9,True
4778,4779,286865a9-889c-46d5-96d7-a9d7ec52e64c,54a5c7e0-6531-43c0-8b4b-4eddcf4e46cb,owner,2025-01-20 13:08:02+00:00,54a5c7e0-6531-43c0-8b4b-4eddcf4e46cb,True
4779,4780,187fdb20-83b9-4cce-81fe-04fc4c28e6d0,63d31104-1e07-48f5-8968-f16e2126d20c,owner,2025-01-27 13:04:44+00:00,63d31104-1e07-48f5-8968-f16e2126d20c,True
4780,4781,42b05ea4-22b0-44ff-a0ea-21de0ecd35d8,bab80c4c-74be-454e-a57a-4d41dca102f9,owner,2025-01-27 13:08:10+00:00,bab80c4c-74be-454e-a57a-4d41dca102f9,True


In [8]:
df['is_indexed'].value_counts()


is_indexed
False    3683
Name: count, dtype: int64

In [9]:
df['type'].value_counts()


type
meeting    3683
Name: count, dtype: int64

In [30]:
processor = ContentProcessor(
            qdrant_engine=self.qdrant_engine,
            es_engine=self.es_engine
        )

NameError: name 'self' is not defined